| [Table of Contents](#table_of_contents) | [Data and model](#data_and_model) | [Natural estimators](#natural_estimators) |  [NN-DOOLSE, MLE](#doolse) | [NN-MDOOLSE, REMLE](#mdoolse) | [References](#references) |

**Authors:** Jozef Hanč, Martina Hančová, Andrej Gajdoš  <br> *[Faculty of Science](https://www.upjs.sk/en/faculty-of-science/?prefferedLang=EN), P. J. Šafárik University in Košice, Slovakia* <br> emails: [martina.hancova@upjs.sk](mailto:martina.hancova@upjs.sk)
***
**<font size=6 color=brown> EBLUP-NE for electricity consumption 1</font>** 

<font size=4> Python-based computational tools - </font>  **<font size=4>SageMath</font>**  
<a id=table_of_contents></a>
###  Table of Contents 
* [Data and model](#data_and_model) - data and model description of empirical data
* [Natural estimators](#natural_estimators) - EBLUPNE based on NE
* [NN-DOOLSE, MLE](#doolse) - EBLUPNE based on nonnegative DOOLSE (same as MLE)
* [NN-MDOOLSE, REMLE](#mdoolse) - EBLUPNE based on nonnegative MDOOLSE (same as REMLE)
* [References](#references)

**To get back to the contents, use <font color=brown>the Home key</font>.**

## SageMath - precision and functions
SageMath is a free Python-based mathematics software, an open source alternative to the well-known commercial 
computer algebra systems Mathematica or Maple, which is built on top of SciPy ecosystem, Maxima, R, GAP, FLINT and many others open-source packages; http://www.sagemath.org/

In [1]:
# arbitrary precision in dps decimal places for numerical results 
dps = 20

# precision in bits, real (floating-point) numbers with given precision  
bits = round((dps+1)*ln(10)/ln(2))
RRk = RealField(bits)

# simplifying any symbolic matrix, vector
def simplify_matrix(A): return A.apply_map(lambda item: item.expand())

# converting symbolic matrix, vector to numerical in RRk
def RRk_matrix(A): return A.apply_map(RRk)

from itertools import product

***
<a id=data_and_model></a>
# <font color=brown>Data and Toy model 1</font>

In this FDSLRM application, we model the econometric time series data set, representing the hours observations of the consumption of the electric energy in some department store. The number of time series observations is $n=24$. The data was adapted from _Štulajter & Witkovský, 2004_ and the FDSLRM model from _Gajdoš et al 2017_.

The consumption data can be fitted by the following Gaussian orthogonal FDSLRM:

$ 
\begin{array}{rl}
& X(t) & \!  = \! & \beta_1+\beta_2\cos\left(\tfrac{2\pi t}{24}\right)+\beta_3\sin\left(\tfrac{2\pi t}{24}\right) +\\
&      &         & +Y_1\cos\left(\tfrac{2\pi t\cdot 3}{24}\right)+Y_2\sin\left(\tfrac{2\pi t\cdot 3}{24}\right)+\\
&      &         & +Y_3\cos\left(\tfrac{2\pi t\cdot 4}{24}\right)+Y_4\sin\left(\tfrac{2\pi t\cdot 4}{24}\right)
+w(t), \, t\in \mathbb{N},
\end{array}
$ 

where $\boldsymbol{\beta}=(\beta_1,\,\beta_2,\,\beta_3)' \in \mathbb{R}^3\,,\mathbf{Y} = (Y_1, Y_2, Y_3, Y_4)' \sim \mathcal{N}_4(\boldsymbol{0}, \mathrm{D})\,, w(t) \sim \mathcal{iid}\, \mathcal{N} (0, \sigma_0^2)\,, \boldsymbol{\nu}= (\sigma_0^2, \sigma_1^2, \sigma_2^2, \sigma_3^2, \sigma_4^2) \in \mathbb{R}_{+}^5.$ 

In [2]:
# data - time series observation x
data = [40.3,40.7,38.5,37.9,38.6,41.1,45.2,45.7,46.7,46.5,
        45.2,45.1,45.8,46.3,47.5,48.5,49.1,51.7,50.6,48.0,
        44.7,41.2,40.0,40.3]

# observation x as a one-column matrix of rational numbers (infinite precision)
x = matrix(QQ, data).T
xc = x.T
xc

[403/10 407/10   77/2 379/10  193/5 411/10  226/5 457/10 467/10   93/2  226/5 451/10  229/5 463/10   95/2   97/2 491/10 517/10  253/5     48 447/10  206/5     40 403/10]

In [3]:
# model parameters
n, k, l = 24, 3, 4

# significant frequencies
om1, om2, om3 = 2*pi/24, 2*pi/8, 2*pi/6

# model design matrices F', F, V',V
Fc = matrix([[1 for t in range(1,n+1)],
             [cos(om1*t) for t in range(1,n+1)],
             [sin(om1*t) for t in range(1,n+1)]])
Vc = matrix([[cos(om2*t) for t in range(1,n+1)],
             [sin(om2*t) for t in range(1,n+1)],
             [cos(om3*t) for t in range(1,n+1)],
             [sin(om3*t) for t in range(1,n+1)]])
F, V = Fc.T, Vc.T

# columns vj of V and their squared norm ||vj||^2
vv = lambda j: V[:,j-1]
nv2 = lambda j: (vv(j).T*vv(j)).trace().expand()

In [4]:
# auxiliary matrices and vectors

# Gram matrices GF, GV
GF, GV = (Fc*F).expand(), (Vc*V).expand()
InvGF, InvGV = (GF^(-1)).expand(), (GV^(-1)).expand()

# projectors PF, MF, PV, MV
In = identity_matrix(n)
PF = (F*InvGF*Fc).canonicalize_radical().expand()
PV = (V*InvGV*Vc).canonicalize_radical().expand()
MF, MV = (In-PF).expand(), (In-PV).expand()

# residuals e, e'
e = MF*x
ec = e.T

In [5]:
# orthogonality condition
show((Fc*V).expand())
show(GV)

[0 0 0 0]
[0 0 0 0]
[0 0 0 0]

[12  0  0  0]
[ 0 12  0  0]
[ 0  0 12  0]
[ 0  0  0 12]

***
<a id=natural_estimators></a>
# <font color=brown> Natural estimators</font>

## ANALYTICALLY 
using formula (4.1) from _Hancova et al 2019_

>$
\renewcommand{\arraystretch}{1.4}
\breve{\boldsymbol{\nu}}(\mathbf{e}) =
\begin{pmatrix}
\tfrac{1}{n-k-l}\,\mathbf{e}'\,\mathrm{M_V}\,\mathbf{e} \\
(\mathbf{e}'\mathbf{v}_1)^2/||\mathbf{v}_1||^4 \\
\vdots \\
(\mathbf{e}'\mathbf{v}_l)^2/||\mathbf{v}_l||^4
\end{pmatrix} 
$

## $\boldsymbol{1^{st}}$ stage of EBLUP-NE 

In [6]:
# NE according to formula (4.1)
NE0 = [1/(n-k-l)*(ec*MV*e).trace()]
NEj = [((ec*vv(j)).trace()/nv2(j))^2 for j in [1..l]] 
NE = vector(NE0+NEj)

In [7]:
# final NE
NEsimp = NE.column().expand()
show(NEsimp)

[-6569/4080*sqrt(3)*sqrt(2) - 46513/20400*sqrt(3) - 22533/6800*sqrt(2) + 13147/816]
[                                                      21/160*sqrt(2) + 2683/14400]
[                                                    4429/7200*sqrt(2) + 4769/4800]
[                                                                            1/225]
[                                                                          507/400]

In [8]:
# numerical results with given precision in dps
NEnum = RRk_matrix(NEsimp)
NEnum

[   3.5323140972047290984]
[  0.37193497450591316960]
[   1.8634794260764497182]
[0.0044444444444444444444]
[   1.2675000000000000000]

In [9]:
NEnum.norm()

4.206508062641579

## $\boldsymbol{2^{nd}}$ stage of EBLUP-NE
using formula (3.10) from _Hancova et al 2019_.
>$
\mathring{\nu}_j = \rho_j^2 \breve{\nu}_j; j = 0,1 \ldots, l\\
\rho_0 = 1, \rho_j = \dfrac{\hat{\nu}_j||\mathbf{v}_j||^2}{\hat{\nu}_0+\hat{\nu}_j||\mathbf{v}_j||^2} 
$
>
>where $\boldsymbol{\breve{\nu}}$ are NE,  $\boldsymbol{\hat{\nu}}$ are initial estimates for EBLUP-NE

In [10]:
# EBLUP-NE based on formula (3.9)
rho2 = lambda est: vector( [1] + [ (est[j]*nv2(j)/(est[0]+est[j]*nv2(j)))^2 for j in [1..l] ])
EBLUPNE = lambda est: vector( rho2(est).pairwise_product(NE).list() )

In [11]:
# final EBLUPNE based on NE
EBLUPNEsimp = EBLUPNE(NE).column().expand()
show(EBLUPNEsimp)

[                                                                                                                                                     -6569/4080*sqrt(3)*sqrt(2) - 46513/20400*sqrt(3) - 22533/6800*sqrt(2) + 13147/816]
[           -58140346341/160*sqrt(2)/(7095769382*sqrt(3)*sqrt(2) + 10052804072*sqrt(3) + 5794927786*sqrt(2) - 51856417125) - 22200174206443/43200/(7095769382*sqrt(3)*sqrt(2) + 10052804072*sqrt(3) + 5794927786*sqrt(2) - 51856417125)]
[-39474566680703/288*sqrt(2)/(45287665522*sqrt(3)*sqrt(2) + 64103843304*sqrt(3) - 104085851666*sqrt(2) - 353799573231) - 310322553535051/1600/(45287665522*sqrt(3)*sqrt(2) + 64103843304*sqrt(3) - 104085851666*sqrt(2) - 353799573231)]
[                                                                                                                                      -591872/675/(2562277816*sqrt(3)*sqrt(2) + 3632562703*sqrt(3) + 5902830194*sqrt(2) - 21807671238)]
[                                                                                                                            -3050750840787/50/(17842376068*sqrt(3)*sqrt(2) + 25279321657*sqrt(3) + 38609830986*sqrt(2) - 215185497470)]

In [12]:
# numerical results
rho2(NEnum).column()

[                        1]
[   0.31160298070636086388]
[   0.74578072518084832926]
[0.00022123930617640118797]
[   0.65858525704640917570]

In [13]:
EBLUPNEnum = RRk_matrix(EBLUPNEsimp)
EBLUPNEnum 

[   3.5323140972047290984]
[  0.11589604668498688117]
[   1.3897470377388857171]
[9.8328580522844972432e-7]
[  0.83475681330632363020]

In [14]:
EBLUPNEnum.norm()

3.88830175542161

***
<a id=doolse></a>
# <font color=brown> NN-DOOLSE or MLE</font>

## $\boldsymbol{1^{st}}$ stage of EBLUP-NE 

## KKT algorithm
using the the KKT algorithm (tab.3, _Hancova et al 2019_)  
<img src='KKTscheme.png' width=550 align='left'>  

$~$
>$
\qquad \mathbf{q} = 
\left(\begin{array}{c}
\mathbf{e}'  \mathbf{e}\\
(\mathbf{e}'   \mathbf{v}_{1})^2 \\
\vdots \\
(\mathbf{e}'   \mathbf{v}_{l})^2
\end{array}\right)
$
>
> $\qquad\mathrm{G} = \left(\begin{array}{ccccc}
\small
n^*                  & ||\mathbf{v}_{1}||^2 & ||\mathbf{v}_{2}||^2 & \ldots & ||\mathbf{v}_{l}||^2 \\
||\mathbf{v}_{1}||^2 & ||\mathbf{v}_{1}||^4 & 0                    & \ldots & 0 \\
||\mathbf{v}_{2}||^2 & 0                    & ||\mathbf{v}_{2}||^4 & \ldots & 0 \\
\vdots               & \vdots               & \vdots               & \ldots & \vdots \\
||\mathbf{v}_{l}||^2 & 0                    & 0                    & \ldots & ||\mathbf{v}_{l}||^4
\end{array}\right)
$

In [15]:
# Input: form G
ns = n
u, v, Q  = ns, matrix([nv2(j) for j in [1..l]]), diagonal_matrix([nv2(j)^2 for j in [1..l]])
G = block_matrix([[u,v],[v.T,Q]])

In [16]:
# form q
e2, Ve2 = ec*e, (Vc*e).elementwise_product(Vc*e)
q = e2.stack(Ve2).expand()

In [17]:
# body of the algorithm
for b in product([0,1], repeat=l+1): 
    # set the KKT-conditions matrix K
    K = G*1
    for j in range(l+1): 
        if b[j] == 1: K[0,j], K[j,j]  = 0,-1
    # calculate the auxiliary vector g
    g = K^(-1)*q
    # test non-negativity g
    if RRk(min(g.list()))>=0: break   

In [18]:
# Output: Form estimates nu
nu = g*1
for j in range(l+1):
    if b[j] == 1: nu[j] = 0

In [19]:
# final DOOLSE
DOOLSE = nu
show(DOOLSE)

[  -6569/5040*sqrt(3)*sqrt(2) - 46513/25200*sqrt(3) - 1073/400*sqrt(2) + 328739/25200]
[ 6569/60480*sqrt(3)*sqrt(2) + 46513/302400*sqrt(3) + 1703/4800*sqrt(2) - 68099/75600]
[6569/60480*sqrt(3)*sqrt(2) + 46513/302400*sqrt(3) + 12077/14400*sqrt(2) - 7073/75600]
[                                                                                   0]
[6569/60480*sqrt(3)*sqrt(2) + 46513/302400*sqrt(3) + 1073/4800*sqrt(2) + 54553/302400]

In [20]:
# numerical results
DOOLSEnum = RRk_matrix(DOOLSE)
print(DOOLSEnum, DOOLSE.norm(),b)

([ 2.8620320469435108574]
[0.13343230392728726481]
[ 1.6249767554978238134]
[0.00000000000000000000]
[ 1.0289973294213740952], 3.4508573684415826, (0, 0, 0, 1, 0))


## $\boldsymbol{2^{nd}}$ stage of EBLUP-NE

In [21]:
# final EBLUPNE based on DOOLSE
EBLUPNEsimp = EBLUPNE(DOOLSE).column().expand()
show(EBLUPNEsimp)

[                                                                                                                                                                               -6569/4080*sqrt(3)*sqrt(2) - 46513/20400*sqrt(3) - 22533/6800*sqrt(2) + 13147/816]
[        -21241171347749/3175200*sqrt(3)*sqrt(2)/(10141740*sqrt(2) + 14342689) - 15019912056097/1587600*sqrt(3)/(10141740*sqrt(2) + 14342689) + 3167509606099/264600*sqrt(2)/(10141740*sqrt(2) + 14342689) + 21866439225581/1270080/(10141740*sqrt(2) + 14342689)]
[41628011575877/453600*sqrt(3)*sqrt(2)/(253462812*sqrt(2) + 361618577) + 206026395726467/1587600*sqrt(3)/(253462812*sqrt(2) + 361618577) + 74204155155701/396900*sqrt(2)/(253462812*sqrt(2) + 361618577) + 129704492788567/423360/(253462812*sqrt(2) + 361618577)]
[                                                                                                                                                                                                                                                               0]
[                                                                                                                                     1234006393/14488437600*sqrt(3)*sqrt(2) + 6978001999/57953750400*sqrt(3) + 8156693/57153600*sqrt(2) + 6269605267/28976875200]

In [22]:
# numerical results
rho2(DOOLSEnum).column()

[                     1]
[0.12870278135360510016]
[0.76040524092632442211]
[                     0]
[0.65907154135208143391]

In [23]:
EBLUPNEnum = RRk_matrix(EBLUPNEsimp)
EBLUPNEnum 

[  3.5323140972047290984]
[0.047869065701593229783]
[  1.4169995219469115088]
[ 0.00000000000000000000]
[ 0.83537317866376321748]

In [24]:
EBLUPNEnum.norm()

3.8968282386565334

***
<a id=mdoolse></a>
# <font color=brown> NN-MDOOLSE or REMLE</font>
using the KKT algorithm (tab.3, _Hancova et al 2019_)

## $\boldsymbol{1^{st}}$ stage of EBLUP-NE 

In [25]:
# Input: form G
ns = n-k
u, v, Q  = ns, matrix([nv2(j) for j in [1..l]]), diagonal_matrix([nv2(j)^2 for j in [1..l]])
G = block_matrix([[u,v],[v.T,Q]])

In [26]:
# form q
e2, Ve2 = ec*e, (Vc*e).elementwise_product(Vc*e)
q = e2.stack(Ve2).expand()

In [27]:
# body of the algorithm
for b in product([0,1], repeat=l+1): 
    # set the KKT-conditions matrix K
    K = G*1
    for j in range(l+1): 
        if b[j] == 1: K[0,j], K[j,j]  = 0,-1
    # calculate the auxiliary vector g
    g = K^(-1)*q
    # test non-negativity g
    if RRk(min(g.list()))>=0: break   

In [28]:
# Output: Form estimates nu
nu = g*1
for j in range(l+1):
    if b[j] == 1: nu[j] = 0

In [29]:
# final MDOOLSE
MDOOLSE = nu
show(MDOOLSE)

[  -6569/4320*sqrt(3)*sqrt(2) - 46513/21600*sqrt(3) - 7511/2400*sqrt(2) + 328739/21600]
[6569/51840*sqrt(3)*sqrt(2) + 46513/259200*sqrt(3) + 11291/28800*sqrt(2) - 56089/51840]
[ 6569/51840*sqrt(3)*sqrt(2) + 46513/259200*sqrt(3) + 2803/3200*sqrt(2) - 71213/259200]
[                                                                                    0]
[  6569/51840*sqrt(3)*sqrt(2) + 46513/259200*sqrt(3) + 7511/28800*sqrt(2) - 203/259200]

In [30]:
# numerical results
MDOOLSEnum = RRk_matrix(nu)
print(MDOOLSEnum, MDOOLSE.norm(),b)

([  3.3390373881007626670]
[0.093681858830849614018]
[  1.5852263104013861626]
[ 0.00000000000000000000]
[ 0.98924688432493644442], 3.827466371262348, (0, 0, 0, 1, 0))


## $\boldsymbol{2^{nd}}$ stage of EBLUP-NE

In [31]:
# final EBLUPNE based on NE
EBLUPNEsimp = EBLUPNE(MDOOLSE).column().expand()
show(EBLUPNEsimp)

[                                                                                                                                                                             -6569/4080*sqrt(3)*sqrt(2) - 46513/20400*sqrt(3) - 22533/6800*sqrt(2) + 13147/816]
[            -1504349779139/145800*sqrt(3)*sqrt(2)/(10141740*sqrt(2) + 14342689) - 6807964258873/466560*sqrt(3)/(10141740*sqrt(2) + 14342689) + 698763601463/38880*sqrt(2)/(10141740*sqrt(2) + 14342689) + 30101932999183/1166400/(10141740*sqrt(2) + 14342689)]
[27549505902847/291600*sqrt(3)*sqrt(2)/(253462812*sqrt(2) + 361618577) + 311642980476091/2332800*sqrt(3)/(253462812*sqrt(2) + 361618577) + 92685272269117/583200*sqrt(2)/(253462812*sqrt(2) + 361618577) + 22042322500769/77760/(253462812*sqrt(2) + 361618577)]
[                                                                                                                                                                                                                                                             0]
[                                                                                                                                     196097797/2661141600*sqrt(3)*sqrt(2) + 4431136171/42578265600*sqrt(3) + 4506347/41990400*sqrt(2) + 5525608117/21289132800]

In [32]:
# numerical results
rho2(MDOOLSEnum).column()

[                      1]
[0.063442013996488589587]
[ 0.72365795262659512283]
[                      0]
[ 0.60913484862207232793]

In [33]:
EBLUPNEnum = RRk_matrix(EBLUPNEsimp)
EBLUPNEnum 

[  3.5323140972047290984]
[0.023596303858387770044]
[  1.3485217062362661184]
[ 0.00000000000000000000]
[ 0.77207842062847667565]

In [34]:
EBLUPNEnum.norm()

3.859069259116448

***
<a id=references></a>
# <font color=brown> References </font>
This notebook belongs to suplementary materials of the paper submitted to Statistical Papers and available at <https://arxiv.org/abs/1905.07771>.

* Hančová, M., Vozáriková, G., Gajdoš, A., Hanč, J. (2019). [Estimating variance components in time series
	linear regression models using empirical BLUPs and convex optimization](https://arxiv.org/abs/1905.07771), https://arxiv.org/, 2019.

### Abstract of the paper

We propose a two-stage estimation method of variance components in time series models known as FDSLRMs, whose observations can be described by a linear mixed model (LMM). We based estimating variances, fundamental quantities in a time series forecasting approach called kriging, on the empirical (plug-in) best linear unbiased predictions of unobservable random components in FDSLRM. 

The method, providing invariant non-negative quadratic estimators, can be used for any absolutely continuous probability distribution of time series data. As a result of applying the convex optimization and the LMM methodology, we resolved two problems $-$ theoretical existence and equivalence between least squares estimators, non-negative (M)DOOLSE, and maximum likelihood estimators, (RE)MLE, as possible starting points of our method and a 
practical lack of computational implementation for FDSLRM. As for computing (RE)MLE in the case of $ n $ observed time series values, we also discovered a new algorithm of order $\mathcal{O}(n)$, which at the default precision is $10^7$ times more accurate and $n^2$ times faster than the best current Python(or R)-based computational packages, namely CVXPY, CVXR, nlme, sommer and mixed. 

We illustrate our results on three real data sets $-$ electricity consumption, tourism and cyber security $-$ which are easily available, reproducible, sharable and modifiable in the form of interactive Jupyter notebooks.

* Gajdoš, A., Hančová, M., Hanč, J. (2017). [Kriging Methodology and Its Development in Forecasting Econometric Time Series](https://www.czso.cz/csu/czso/statistika-statistics-and-economy-journal-no-12017). _Statistica: Statistics and Economy Journal_, 2017, Vol. 97, No. 1, pp. 59–73
$~$
* Štulajter, F., Witkovský, V. (2004). [Estimation of Variances in Orthogonal Finite
Discrete Spectrum Linear Regression Models](https://link.springer.com/article/10.1007/s001840300299). _Metrika_, 2004, Vol. 60, No. 2, pp. 105–118

| [Table of Contents](#table_of_contents) | [Data and model](#data_and_model) | [Natural estimators](#natural_estimators) |  [NN-DOOLSE, MLE](#doolse) | [NN-MDOOLSE, REMLE](#mdoolse) | [References](#references) |